In [ ]:
#from urllib.request import urlretrieve
#urlretrieve("http://www.eigenvector.com/data/Corn/corn.mat", "corn.mat")

In [ ]:
import pandas
import numpy
from matplotlib import pyplot
# load data
from scipy.io import loadmat
corn = loadmat("corn.mat")
prop = pandas.DataFrame(corn['propvals']['data'][0][0], columns=corn['propvals'][0][0][8][1][0])
data = pandas.DataFrame(corn['m5spec']['data'][0][0], columns=corn['m5spec'][0][0][9][1][0][0])
data.index = prop.iloc[:, 1].values
data.T.plot()
pyplot.gca().legend_ = None
pyplot.show()
# SNV
data = ((data.T - data.T.mean()) / data.T.std()).T
# 2nd derivative
window = 7
from scipy.signal import savgol_filter
buff = data.values
buff = savgol_filter(buff, window, 2, 0)
buff = savgol_filter(buff, window, 2, 1)
buff = savgol_filter(buff, window, 2, 0)
buff = savgol_filter(buff, window, 2, 1)
data = pandas.DataFrame(buff, index=data.index, columns=data.columns)
# split data
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, train_size=0.6, random_state=8)
pyplot.violinplot([data.index, train.index, test.index])
pyplot.show()

In [ ]:
# PLS（モデリングだけ）
from sklearn.cross_decomposition import PLSRegression
model = PLSRegression().fit(data, data.index)
calibration = model.predict(data)
pyplot.figure(figsize=(4, 4))
pyplot.plot([data.index.min(), data.index.max()], [data.index.min(), data.index.max()])
pyplot.scatter(data.index, calibration)
from sklearn.metrics import root_mean_squared_error, r2_score
print("RMSEC =", root_mean_squared_error(data.index, calibration))
print("R^2 =", r2_score(data.index, calibration))

In [ ]:
# PCR
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
p1 = numpy.arange(2, 31, 1)
parm = {"pca__n_components": p1}
pipe = make_pipeline(PCA(), LinearRegression())
search = GridSearchCV(pipe, parm).fit(data, data.index)
print(search.best_estimator_, search.best_score_)
pyplot.scatter(p1, search.cv_results_["mean_test_score"])
pyplot.show()
model = search.best_estimator_.fit(train, train.index)

In [ ]:
# PLS
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV
p1 = numpy.arange(2, 31, 1)
parm = {"n_components": p1}
search = GridSearchCV(PLSRegression(), parm).fit(data, data.index)
print(search.best_estimator_, search.best_score_)
pyplot.scatter(p1, search.cv_results_["mean_test_score"])
pyplot.show()
model = search.best_estimator_.fit(train, train.index)

In [ ]:
# PLS leave-one-out cross varidation
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import r2_score

model = PLSRegression(n_components=3)
r2_scores = []
for train_index, test_index in LeaveOneOut().split(data):
  model.fit(data.values[train_index], data.index.values[train_index])
  buff = r2_score(data.index.values[train_index], model.predict(data.values[train_index]))
  r2_scores.append(buff)
average_r2 = numpy.mean(r2_scores)
print("R^2 =", average_r2)

In [ ]:
# model.coef_ による予測
spec = test.iloc[0].values
print(model.predict([spec])[0])
print(((spec - model._x_mean)  @ model.coef_.T + model._y_mean)[0])

In [ ]:
# Ridge
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
p1 = numpy.logspace(-9, -6)
parm = {"alpha": p1}
search = GridSearchCV(Ridge(), parm).fit(data, data.index)
print(search.best_estimator_, search.best_score_)
pyplot.scatter(p1, search.cv_results_["mean_test_score"])
pyplot.show()
model = search.best_estimator_.fit(train, train.index)

In [ ]:
# Lasso
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
p1 = numpy.logspace(-9, -6)
parm = {"alpha": p1}
search = GridSearchCV(Lasso(), parm).fit(data, data.index)
print(search.best_estimator_, search.best_score_)
pyplot.scatter(p1, search.cv_results_["mean_test_score"])
pyplot.show()
model = search.best_estimator_.fit(train, train.index)

In [ ]:
# SVM
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
grid = 10
p1 = numpy.logspace(3, 5, num=grid)
p2 = numpy.logspace(0, 2, num=grid)
parm = {"C": p1, "gamma": p2}
search = GridSearchCV(SVR(kernel="rbf"), parm).fit(data, data.index)
map = pandas.DataFrame(search.cv_results_["mean_test_score"].reshape(grid, grid), index=p1, columns=p2)
import seaborn
seaborn.heatmap(map)
pyplot.ylabel("C")
pyplot.xlabel("gamma")
pyplot.show()
print(search.best_estimator_, search.best_score_)
model = search.best_estimator_.fit(train, train.index)

In [ ]:
# calibration and validation
calibration = model.predict(train)
validation = model.predict(test)
from sklearn.metrics import root_mean_squared_error, r2_score
print("RMSEC =", root_mean_squared_error(train.index, calibration))
print("R^2 =", r2_score(train.index, calibration))
print("")
print("RMSEV =", root_mean_squared_error(test.index, validation))
print("R^2 =", r2_score(test.index, validation))
print("")
pyplot.figure(figsize=(4, 4))
pyplot.plot([data.index.min(), data.index.max()], [data.index.min(), data.index.max()])
pyplot.scatter(train.index, calibration)
pyplot.scatter(test.index, validation)
pyplot.show()